In [28]:
from bs4 import BeautifulSoup
import requests

link = 'https://cs.hse.ru/persons'
soup = BeautifulSoup(requests.get(link).text, 'lxml')

In [29]:
print(len(soup.find_all('a')))
soup.find_all('a', {'class':'link'})[:5]

2578


[<a class="link link_no-visited link_no-underline" href="https://cs.hse.ru/persons/">RU</a>,
 <a class="link link_no-visited link_no-underline" href="https://cs.hse.ru/en/persons/">EN</a>,
 <a class="link link_dark no-visited" href="//www.hse.ru/search/search.html?simple=0">Расширенный поиск по сайту</a>,
 <a class="link header_breadcrumb__link" href="//www.hse.ru">
 <span>Национальный исследовательский университет «Высшая школа экономики»</span>
 </a>,
 <a class="link header_breadcrumb__link" href="http://www.hse.ru/education/faculty/">
 <span>Факультеты</span>
 </a>]

In [30]:
persons_links = set()

for link in soup.find_all('a'):
    try:
        link_href = link.get('href')
        if (('staff' in link_href) or ('persons' in link_href)) and ('?' not in link_href):
            persons_links.add(link.get('href'))
    except:
        pass

In [31]:
len(persons_links)

378

In [36]:
persons_list = list(persons_links)
persons_list = ['http://' + elem.strip('//') for elem in persons_list]
print(persons_list)

['http://www.hse.ru/org/persons/191823857', 'http://www.hse.ru/org/persons/225575569', 'http://www.hse.ru/org/persons/7103512', 'http://www.hse.ru/org/persons/307094510', 'http://https://cs.hse.ru/hdilab/persons', 'http://www.hse.ru/org/persons/14266660', 'http://www.hse.ru/org/persons/191193864', 'http://www.hse.ru/org/persons/207133933', 'http://www.hse.ru/org/persons/218368371', 'http://www.hse.ru/org/persons/137291612', 'http://www.hse.ru/org/persons/68153', 'http://www.hse.ru/staff/leyla', 'http://www.hse.ru/org/persons/309076802', 'http://www.hse.ru/org/persons/228364473', 'http://www.hse.ru/org/persons/68054814', 'http://www.hse.ru/org/persons/305072987', 'http://www.hse.ru/staff/arazorenova', 'http://www.hse.ru/org/persons/148813333', 'http://www.hse.ru/org/persons/307095000', 'http://www.hse.ru/org/persons/221998534', 'http://www.hse.ru/org/persons/161005930', 'http://www.hse.ru/staff/anaumov', 'http://www.hse.ru/staff/a_levin', 'http://www.hse.ru/staff/lzhukov', 'http://www.h

In [42]:
import time
import random
cv = {}
for link in persons_list[:20]:
    time.sleep(random.randint(1,5))
    try:
        link_text = requests.get(link).text
        if 'Резюме' in link_text:
            soup = BeautifulSoup(link_text, 'lxml')
            key = re.findall('Сотрудники - ([\w\s]+)', soup.find_all('title')[0].get_text())[0].strip()
            value = 'https://www.hse.ru/' + soup.find_all('a', text='Резюме')[0].get('href')
            cv[key] = value
    except:
        pass

print(cv)


{'Попцова Мария Сергеевна': 'https://www.hse.ru//data/2020/01/11/365040347054515/1MariaPoptsovaCV_2019.pdf', 'Сапарина Ирина Олеговна': 'https://www.hse.ru//data/2018/12/18/12700327357033/CV_ru.pdf', 'Бабенко Максим Александрович': 'https://www.hse.ru//data/2014/02/21/1241147082/CVBabenko.pdf', 'Кодрян Максим Станиславович': 'https://www.hse.ru//data/2019/09/24/175182767468342/%D0%9A%D0%BE%D0%B4%D1%80%D1%8F%D0%BD_%D1%80%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_%D1%80%D1%83%D1%81.pdf', 'Яковлева Илона Александровна': 'https://www.hse.ru//data/2020/08/30/294040652059594/%D0%AF%D0%BA%D0%BE%D0%B2%D0%BB%D0%B5%D0%B2%D0%B0%20%D0%98%D0%BB%D0%BE%D0%BD%D0%B0%20(1).pdf', 'Пучкин Никита Андреевич': 'https://www.hse.ru//data/2018/06/03/106820579200462/puchkin_cv_rus.pdf', 'Гадецкий Артем Валерьевич': 'https://www.hse.ru//data/2019/03/12/310912501447694/CV.pdf', 'Колесниченко Елена Юрьевна': 'https://www.hse.ru//data/2019/11/29/125140360151399/%D0%A0%D0%B5%D0%B7%D1%8E%D0%BC%D0%B5_%D0%9A%D0%BE%D0%BB%D0%B5%D1%81

In [48]:
for key,value in cv.items():
    doc = requests.get(value, stream=True) # потоковое чтение файла, потому pdf может быть большим и не уместиться в памяти
    if doc.headers['content-type'] == 'application/pdf': # на всякий случай делаем проверку, иначе получим битый файл
        fh = open(f'{key}.pdf', 'wb') # wb - запись байтовой информации
        fh.write(doc.content) # считываем туда "содержание" файла по ссылке
        fh.close()
    

In [65]:
languages = {}
for link in persons_list:
    time.sleep(random.randint(1,5))
    try:
        link_text = requests.get(link).text
        soup = BeautifulSoup(link_text, 'lxml')
        langs = soup.find_all('dl', {'class':"main-list large main-list-language-knowledge-level"})[0].find_all("dd")
        langs = [lang.get_text() for lang in langs]
        for lang in langs:
            languages[lang] = languages.get(lang, 0) + 1


    except:
        pass
    


KeyboardInterrupt: 

In [66]:
languages

{'французский': 20,
 'английский': 112,
 'русский': 19,
 'немецкий': 13,
 'испанский': 3,
 'украинский': 2,
 'китайский': 1,
 'нидерландский': 1,
 'хинди': 1,
 'сербский': 2,
 'английский (Читаю, перевожу, пишу научные статьи)': 1,
 'английский (IELTS 7)': 1,
 'итальянский': 1,
 'английский (свободный)': 1,
 'английский (Свободное владение)': 1,
 'немецкий (Средний уровень)': 1,
 'французский (Средний уровень)': 1,
 'исландский': 1,
 'японский': 1}

In [62]:
print(link)
link_text = requests.get(link).text
soup = BeautifulSoup(link_text, 'lxml')
langs = soup.find_all('dl', {'class':"main-list large main-list-language-knowledge-level"})[0].find_all("dd")
langs = [lang.get_text() for lang in langs]
langs

http://www.hse.ru/org/persons/191823857


['французский', 'английский']